In [2]:
import yfinance as yf

In [3]:
aapl = yf.Ticker("AAPL")

In [4]:
data = yf.download("AAPL", interval="1d", start="1999-12-31", end="2014-12-31")

[*********************100%***********************]  1 of 1 completed


In [6]:
data["Close"]

Date
1999-12-30     0.895647
1999-12-31     0.917969
2000-01-03     0.999442
2000-01-04     0.915179
2000-01-05     0.928571
                ...    
2014-12-23    28.135000
2014-12-24    28.002501
2014-12-26    28.497499
2014-12-29    28.477501
2014-12-30    28.129999
Name: Close, Length: 3774, dtype: float64

In [7]:
data = yf.download("YNDX", interval="1d", start="1999-12-31", end="2016-12-31")

[*********************100%***********************]  1 of 1 completed


In [10]:
series = data["Close"]

In [17]:
import pandas as pd
import numpy as np
def transform_to_DOIU(prices):
    arr = pd.DataFrame(prices)[0].pct_change().to_numpy()[1:]
    x = [1]
    for el in arr:
        x.append(x[-1]*(1+el))
    return np.array(x)


In [18]:
tr_series = transform_to_DOIU(series.values)

In [27]:
OUTPUT_WINDOW_SIZE = 30
INPUT_WINDOW_SIZE = 60
prices = series.values

X = []
y = []
for i in range(0, len(prices), OUTPUT_WINDOW_SIZE):

    tr_x = transform_to_DOIU(prices[i:INPUT_WINDOW_SIZE+i
                                    + OUTPUT_WINDOW_SIZE])
    y.append(tr_x[-1])
    X.append(tr_x[:INPUT_WINDOW_SIZE])

X = X[:-1]
y = y[:-1]

In [30]:
len(X), len(y)

(47, 47)

In [102]:
from sklearn.svm import LinearSVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
def prepare_data(prices, OUTPUT_WINDOW_SIZE = 30, INPUT_WINDOW_SIZE = 60):
    X = []
    y = []
    for i in range(0, len(prices)-OUTPUT_WINDOW_SIZE-INPUT_WINDOW_SIZE,
                   OUTPUT_WINDOW_SIZE):

        tr_x = transform_to_DOIU(prices[i:INPUT_WINDOW_SIZE+i
                                        + OUTPUT_WINDOW_SIZE])
        y.append(tr_x[-1].reshape(1, -1))
        X.append(tr_x[:INPUT_WINDOW_SIZE].reshape(1,-1))

    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    return X, y

def train_model(model, X, y, random_state=7575):
    X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                        random_state=random_state)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    return model, mean_squared_error(y_test, y_pred, squared=False)

In [105]:
X, y = prepare_data(series.values)
model = LinearSVR(max_iter=10000)
best_model, rmse = train_model(model, X, y.ravel())

In [106]:
rmse

0.18681286223411178

In [107]:
import pickle
with open('svr.pkl', 'wb') as fid:
    pickle.dump(best_model, fid)

LinearRegression()

0.34744508120041334

In [91]:
y_test

array([[0.71076925],
       [0.53163266],
       [1.04141104],
       [1.04231489],
       [1.03677448],
       [0.80091626],
       [0.62314846],
       [0.67523679],
       [1.07038126],
       [1.5918467 ],
       [1.6571668 ],
       [0.58814514]])

In [92]:
y_pred

array([[0.7266584 ],
       [0.70022339],
       [0.89017808],
       [1.02007216],
       [0.89217876],
       [0.83630119],
       [1.17285766],
       [1.22651197],
       [0.47988839],
       [1.45289927],
       [1.05193647],
       [0.77264184]])